In [ ]:
# Vemos si tenemos instalado CUDA
! dir /usr/local/

bin  colab  cuda  cuda-12  cuda-12.2  etc  games  include  lib	lib64  man  opt  sbin  share  src


In [ ]:
# Para checar la versión del compilador de CUDA
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
# Ejemplo_1 HelloCUDA
%%writefile HelloCUDA.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void helloCUDA(float e){
    printf("Soy el hilo %d del bloque %d con valor e=%f\n",threadIdx.x,blockIdx.x,e);
}

int main(void){
    printf("\nHello World\n");

    helloCUDA<<<4,4>>>(2.5f);

    cudaDeviceReset();
    return(0);
}

Overwriting HelloCUDA.cu


In [ ]:
# Compilando con nvcc
! nvcc HelloCUDA.cu -o HelloCUDA

In [ ]:
# Ejecutando el programa
! ./HelloCUDA


Hello World
Soy el hilo 0 del bloque 1 con valor e=2.500000
Soy el hilo 1 del bloque 1 con valor e=2.500000
Soy el hilo 2 del bloque 1 con valor e=2.500000
Soy el hilo 3 del bloque 1 con valor e=2.500000
Soy el hilo 0 del bloque 3 con valor e=2.500000
Soy el hilo 1 del bloque 3 con valor e=2.500000
Soy el hilo 2 del bloque 3 con valor e=2.500000
Soy el hilo 3 del bloque 3 con valor e=2.500000
Soy el hilo 0 del bloque 2 con valor e=2.500000
Soy el hilo 1 del bloque 2 con valor e=2.500000
Soy el hilo 2 del bloque 2 con valor e=2.500000
Soy el hilo 3 del bloque 2 con valor e=2.500000
Soy el hilo 0 del bloque 0 con valor e=2.500000
Soy el hilo 1 del bloque 0 con valor e=2.500000
Soy el hilo 2 del bloque 0 con valor e=2.500000
Soy el hilo 3 del bloque 0 con valor e=2.500000


In [ ]:
#Ejemplo_2 suma_vectores
%%writefile suma_vectores.cu
/*
Programa:      Suma_Vectores.cu
Descripción:   Dados dos vectores de tamaño N, sumarlos en paralelo y guardar el resultado en un tercer vector.
Actualización: 09/Jul/2020
*/


#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Función Kernel que se ejecuta en el Device.
__global__ void Suma_vectores(float *c_d,float *a_d,float *b_d, int N)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if (idx<N){
	  c_d[idx] = a_d[idx] + b_d[idx];
  }
}

// Código principal que se ejecuta en el Host
int main(void){
	float *a_h,*b_h,*c_h; //Punteros a arreglos en el Host
	float *a_d,*b_d,*c_d;  //Punteros a arreglos en el Device
	const int N = 24;  //Número de elementos en los arreglos  (probar 1000000)

	size_t size=N * sizeof(float);

	a_h = (float *)malloc(size); // Pedimos memoria en el Host
	b_h = (float *)malloc(size);
	c_h = (float *)malloc(size);//También se puede con cudaMallocHost

	//Inicializamos los arreglos a,b en el Host
	srand(time(NULL));
	for (int i=0; i<N; i++){
		//a_h[i] = (float)i;b_h[i] = (float)(i+1);
		a_h[i] = rand() % 100 + 1.0;
		b_h[i] = rand() % 100 + 1.0;
	}

	printf("\nArreglo a:\n");
	for (int i=0; i<N; i++) printf("%f ", a_h[i]);
	printf("\n\nArreglo b:\n");
	for (int i=0; i<N; i++) printf("%f ", b_h[i]);

	cudaMalloc((void **) &a_d,size);   // Pedimos memoria en el Device
	cudaMalloc((void **) &b_d,size);
	cudaMalloc((void **) &c_d,size);

	//Pasamos los arreglos a y b del Host al Device
	cudaMemcpy(a_d, a_h, size, cudaMemcpyHostToDevice);
	cudaMemcpy(b_d, b_h, size, cudaMemcpyHostToDevice);

	//Realizamos el cálculo en el Device
	int block_size =8;
	int n_blocks = N/block_size + (N%block_size == 0 ? 0:1);

	Suma_vectores <<< n_blocks, block_size >>> (c_d,a_d,b_d,N);

	//Pasamos el resultado del Device al Host
	cudaMemcpy(c_h, c_d, size,cudaMemcpyDeviceToHost);

	//Resultado
	printf("\n\nArreglo c:\n");
	for (int i=0; i<N; i++) printf("%f ", c_h[i]);

	printf("\n\nFin del programa...\n");
	//system("pause");

	// Liberamos la memoria del Host
	free(a_h);
	free(b_h);
	free(c_h);

	// Liberamos la memoria del Device
	cudaFree(a_d);
	cudaFree(b_d);
	cudaFree(c_d);
	return(0);
}

Writing suma_vectores.cu


In [ ]:
! nvcc suma_vectores.cu -o suma_vectores

In [ ]:
! ./suma_vectores


Arreglo a:
8.000000 36.000000 32.000000 79.000000 34.000000 43.000000 4.000000 96.000000 39.000000 83.000000 57.000000 22.000000 36.000000 64.000000 52.000000 58.000000 99.000000 59.000000 22.000000 42.000000 27.000000 44.000000 69.000000 87.000000 

Arreglo b:
16.000000 48.000000 78.000000 12.000000 38.000000 89.000000 33.000000 6.000000 8.000000 61.000000 27.000000 24.000000 74.000000 39.000000 32.000000 59.000000 93.000000 31.000000 37.000000 55.000000 37.000000 30.000000 91.000000 7.000000 

Arreglo c:
24.000000 84.000000 110.000000 91.000000 72.000000 132.000000 37.000000 102.000000 47.000000 144.000000 84.000000 46.000000 110.000000 103.000000 84.000000 117.000000 192.000000 90.000000 59.000000 97.000000 64.000000 74.000000 160.000000 94.000000 

Fin del programa...


In [ ]:
#Ejemplo_3 Sumatoria
%%writefile kernel.h
#include <cuda_runtime.h>
#include <stdio.h>
//#include <time.h>
double sumOfArrayGPU(double* A, long int n);


Writing kernel.h


In [ ]:
%%writefile kernel.cu
#include "kernel.h"

#define TPB 1024
#define ATOMIC 1 // 0 para no usar el atomicAdd

__global__ void sumOfArrayKernel(double *d_sum_total, double *d_A, long int n) {
	const long int idx = threadIdx.x + blockDim.x * blockIdx.x;
	const int s_idx = threadIdx.x;
	__shared__ double s_data[TPB];

	s_data[s_idx] = (idx<n) ? d_A[idx] : 0.0;//Util cuando n/TPB no es entero
	__syncthreads();

	if (s_idx==0) {
		double blockSum = 0.0;
		for (int j = 0; j < blockDim.x; j++) {
			blockSum += s_data[j];
		}
		//printf("Block_%d, blockSum = %lf\n", blockIdx.x, blockSum);
		if (ATOMIC) {
			atomicAdd(d_sum_total, blockSum);//Hay que asegurarnos de tener una tarjeta con capacidad >= 6.0 para usar double en el atomicAdd
			                                 //Mi GPU tiene capaciad 5.2, por lo tanto no va a reconocer el atomicAdd con valores double
			                                 //Vamos a probar en la GPU Quadro 8000 que tiene capacidad 7.5
		}
		else {
			*d_sum_total += blockSum; //Resultados no esperados
		}
	}
}


__global__ void sumOfArrayKernel_V2(double* d_sum_total, double* d_A, long int n) {
	const long int idx = threadIdx.x + blockDim.x * blockIdx.x;

	double blockSum = 0.0;
	const int s_idx = threadIdx.x;
	__shared__ double s_data[TPB];

	s_data[s_idx]= blockSum = (idx < n) ? d_A[idx] : 0.0;

	__syncthreads();

	for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1)
	{
		if (s_idx < s)
		{
			s_data[s_idx] = blockSum = blockSum + s_data[s_idx + s];
		}

		__syncthreads();
	}

	if (s_idx == 0) {
		if (ATOMIC) {
			atomicAdd(d_sum_total, blockSum);
		}
		else {
			*d_sum_total += blockSum; //Resultados no esperados
		}
	}
}



double sumOfArrayGPU(double *A, long int n){
	double *d_A;
	double *d_sum_total;
	double sum_total;
	//double t_ini, t_fin,time_kernel;

	//1. Crear memoria en la GPU
	cudaMalloc(&d_sum_total, sizeof(double));
	cudaMalloc(&d_A, n * sizeof(double));

	//Inicializamos en cero
	cudaMemset(d_sum_total, 0, sizeof(double));

	//2. Copiar memoria (CPU-->GPU)
	cudaMemcpy(d_A, A, n * sizeof(double), cudaMemcpyHostToDevice);

	//t_ini = clock();
	//3. Ejecutar función Kernel
	sumOfArrayKernel <<<(n+TPB-1)/TPB,TPB >>> (d_sum_total,d_A,n);
	//sumOfArrayKernel_V2 << <(n + TPB - 1) / TPB, TPB >> > (d_sum_total, d_A, n);
	/*cudaDeviceSynchronize();
	t_fin = clock();
	time_kernel = (t_fin - t_ini) / CLOCKS_PER_SEC;
	printf("\nTiempo de procesamiento del Kernel: %lf seconds\n", time_kernel);
	*/
	//4. Copiar memoria (GPU-->CPU)
	cudaMemcpy(&sum_total, d_sum_total, sizeof(double), cudaMemcpyDeviceToHost);

	cudaFree(d_sum_total);
	cudaFree(d_A);
	cudaDeviceReset();
	return(sum_total);
}

Overwriting kernel.cu


In [ ]:
%%writefile sumatoria_main.cpp
/*
Programa:       sumatoria_main.cpp
Descripción:    Dado un arreglo de elementos en punto flotante (double) de tamaño N
                calcular la sumatoria de todos los elementos del arreglo
Actualización:  07/Jul/2020
*/

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

/*CUDA*/
#include "kernel.h"

double* generateRandomArray(long int numberOfElements);
double sumOfArraySeq(double* A, long int numberOfElements);
void printArray(double* A, long int n);


int main(int argc, char* argv[]) {
    double t_ini, t_fin;
    double time_generateData,time_cpu_seconds,time_gpu_seconds;
    double *A;
    double sum_total_cpu,sum_total_gpu;
    long int n=500000000;

    printf("De cuantos elementos son los arreglos\n");
    scanf("%ld", &n);

    t_ini = clock();
    A = generateRandomArray(n);
    t_fin = clock();
    time_generateData = (t_fin - t_ini) / CLOCKS_PER_SEC;
    //printArray(A,n);
    //
    t_ini = clock();
    sum_total_cpu = sumOfArraySeq(A, n);
    t_fin = clock();
    time_cpu_seconds = (t_fin - t_ini) / CLOCKS_PER_SEC;

    t_ini = clock();
    sum_total_gpu = sumOfArrayGPU(A,n);
    t_fin = clock();
    time_gpu_seconds = (t_fin - t_ini) / CLOCKS_PER_SEC;

    printf("La suma del arreglo en CPU es: %lf \n", sum_total_cpu);
    printf("La suma del arreglo en GPU es: %lf \n", sum_total_gpu);
    printf("Tiempo para generar datos: %lf segundos.\n", time_generateData);
    printf("Tiempo de procesamiento en CPU: %lf segundos.\n",time_cpu_seconds);
    printf("Tiempo de procesamiento en GPU: %lf segundos.\n", time_gpu_seconds);

    free(A);
}

double* generateRandomArray(long int numberOfElements) {
    double* myArray;
    //srand(time(NULL));
    srand(1);
    myArray = (double *)malloc(numberOfElements * sizeof(double));
    for (long int i = 0; i < numberOfElements; i++) {
        myArray[i] = rand() % 100 + 1.0;
        //myArray[i] = 1.0;
    }
    return myArray;
}

void printArray(double* A, long int n) {

    printf("Arreglo=");
    for (long int i = 0; i < n; i++) {
        printf("%f  ", A[i]);
    }
    printf("\n");
}

double sumOfArraySeq(double* A, long int numberOfElements) {
    double resultSum = 0.0;
    for (long int i = 0; i < numberOfElements; i++) {
        resultSum = resultSum + A[i];
    }
    return resultSum;
}

Overwriting sumatoria_main.cpp


In [ ]:
# Compilando usando -arch para especificar la arquitectura de la GPU
! nvcc -arch=compute_75 kernel.cu sumatoria_main.cpp -o run_sumatoria_V1

In [ ]:
# Usando el perfilador (profiler) de nvidia: nvprof
! nvprof ./run_sumatoria_V1

De cuantos elementos son los arreglos
1000000000
==1608== NVPROF is profiling process 1608, command: ./run_sumatoria_V1
La suma del arreglo en CPU es: 50501072177.000000 
La suma del arreglo en GPU es: 50501072177.000000 
Tiempo para generar datos: 23.084276 segundos.
Tiempo de procesamiento en CPU: 3.073492 segundos.
Tiempo de procesamiento en GPU: 2.962195 segundos.
==1608== Profiling application: ./run_sumatoria_V1
==1608== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   64.56%  1.67953s         1  1.67953s  1.67953s  1.67953s  [CUDA memcpy HtoD]
                   35.44%  921.85ms         1  921.85ms  921.85ms  921.85ms  sumOfArrayKernel(double*, double*, long)
                    0.00%  1.9520us         1  1.9520us  1.9520us  1.9520us  [CUDA memset]
                    0.00%  1.8240us         1  1.8240us  1.8240us  1.8240us  [CUDA memcpy DtoH]
      API calls:   88.61%  2.60171s         2  1.30085s  921.88ms  1

In [ ]:
# Asegurarse de descomentar la versión V2:
# //sumOfArrayKernel <<<(n+TPB-1)/TPB,TPB >>> (d_sum_total,d_A,n);
#	sumOfArrayKernel_V2 << <(n + TPB - 1) / TPB, TPB >> > (d_sum_total, d_A, n);

! nvcc -arch=compute_75 kernel.cu sumatoria_main.cpp -o run_sumatoria_V2

In [ ]:
! nvprof ./run_sumatoria_V2

De cuantos elementos son los arreglos
1000000000
==2090== NVPROF is profiling process 2090, command: ./run_sumatoria_V2
La suma del arreglo en CPU es: 50501072177.000000 
La suma del arreglo en GPU es: 50501072177.000000 
Tiempo para generar datos: 22.214245 segundos.
Tiempo de procesamiento en CPU: 3.228597 segundos.
Tiempo de procesamiento en GPU: 2.267603 segundos.
==2090== Profiling application: ./run_sumatoria_V2
==2090== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   87.54%  1.68190s         1  1.68190s  1.68190s  1.68190s  [CUDA memcpy HtoD]
                   12.46%  239.48ms         1  239.48ms  239.48ms  239.48ms  sumOfArrayKernel_V2(double*, double*, long)
                    0.00%  2.1760us         1  2.1760us  2.1760us  2.1760us  [CUDA memcpy DtoH]
                    0.00%  1.9520us         1  1.9520us  1.9520us  1.9520us  [CUDA memset]
      API calls:   85.59%  1.92171s         2  960.85ms  239.51ms